In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

random.seed(777)
torch.manual_seed(777)

In [5]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
drop_prob = 0.5

In [3]:
# MNIST dataset
mnist_train = dsets.MNIST(root = 'data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download = True)

mnist_test = dsets.MNIST(root = 'data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download = True)

In [4]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)

In [6]:
# nn layers
linear1 = torch.nn.Linear(784, 512, bias = True)
linear2 = torch.nn.Linear(512, 512, bias = True)
linear3 = torch.nn.Linear(512, 512, bias = True)
linear4 = torch.nn.Linear(512, 512, bias = True)
linear5 = torch.nn.Linear(512, 10, bias = True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=drop_prob)

In [7]:
# xavier initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[-0.0565,  0.0423, -0.0155,  ...,  0.1012,  0.0459, -0.0191],
        [ 0.0772,  0.0452, -0.0638,  ...,  0.0476, -0.0638,  0.0528],
        [ 0.0311, -0.1023, -0.0701,  ...,  0.0412, -0.1004,  0.0738],
        ...,
        [ 0.0334,  0.0187, -0.1021,  ...,  0.0280, -0.0583, -0.1018],
        [-0.0506, -0.0939, -0.0467,  ..., -0.0554, -0.0325,  0.0640],
        [-0.0183, -0.0123,  0.1025,  ..., -0.0214,  0.0220, -0.0741]],
       requires_grad=True)

In [8]:
# model
model = torch.nn.Sequential(linear1, relu, dropout,
                            linear2, relu, dropout,
                            linear3, relu, dropout,
                            linear4, relu, dropout,
                            linear5)

In [9]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
# training
total_batch = len(data_loader)
model.train() # set the model to train mode(dropout = True)
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        # label is not one-hot encoded
        X = X.view(-1, 28*28)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

Epoch: 0001 cost = 0.485160440
Epoch: 0002 cost = 0.215965435
Epoch: 0003 cost = 0.180188105
Epoch: 0004 cost = 0.156184211
Epoch: 0005 cost = 0.146125466
Epoch: 0006 cost = 0.130428195
Epoch: 0007 cost = 0.125054240
Epoch: 0008 cost = 0.117369540
Epoch: 0009 cost = 0.111367799
Epoch: 0010 cost = 0.106937215
Epoch: 0011 cost = 0.103302673
Epoch: 0012 cost = 0.100896187
Epoch: 0013 cost = 0.097530089
Epoch: 0014 cost = 0.092342444
Epoch: 0015 cost = 0.097249724


In [11]:
# Test the model using test sets
with torch.no_grad():
    model.eval() # set the model to evalutaion mode (dropout = False)
    
    X_test = mnist_test.test_data.view(-1, 28*28).float()
    Y_test = mnist_test.test_labels
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy : ', accuracy.item())

C:\Users\user\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:63: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\user\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:53: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy :  0.9732999801635742
